In [ ]:
import functools
import operator
from matplotlib import pyplot as plt

import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import math
import random
from scipy.optimize import curve_fit
from scipy.stats import poisson

from sklearn.impute import KNNImputer
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.model_selection import train_test_split
import joblib

import seaborn as sns

import sys
sys.path.append('../scripts/')
from config import demographics, vital_sign_vars, lab_vars, treatment_vars, vent_vars, guideline_vars, ffill_windows_clinical, SAMPLE_TIME_H

# Setup
Read file with a selection of patients according to the following criteria:
* age >= 18 years
* ventilated for >= 24 hours
* length of ICU stay >= 72 hours
* known hospital mortality or 90-day-mortality

Data is assumed to be sampled and/or aggregated by periods of 4 hours for the entire ICU stay, i.e. including after and prior to mechanical ventilation.

In [ ]:
file = '../mimic-data/ventilatedpatients.csv'
seed = 12
np.random.seed(seed)
random.seed(seed)

In [ ]:
df = pd.read_csv(file)

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.subject_id.nunique(), df.icustay_id.nunique()

## Preprocessing phase 1
Set some time stamps

In [ ]:
timestamps = [
    'start_time',
    'intime',
    'outtime',
]
for col in timestamps:
    df[col] = pd.to_datetime(df[col])

df = df.sort_values(['icustay_id', 'start_time']).reset_index().drop(['index', ], axis=1)

In [ ]:
def grouped_fill_between(df, colname, fill_value):
    def my_fill_between(group, colname, fill_value):
        group = group.sort_values('start_time')
        from_i = group[colname].first_valid_index()
        to_i = group[colname].last_valid_index()
        return group.loc[from_i:to_i][colname].fillna(fill_value)
    grouped = df.sort_values(['start_time', 'icustay_id']).groupby('icustay_id')
    grouped_filled = grouped.apply(lambda x: my_fill_between(x, colname, fill_value))
    return grouped_filled.reset_index().set_index('level_1')[colname]

df['vent_num_imp'] = grouped_fill_between(df, 'vent_num', 1.0)
df['vent_imp_step'] = df.groupby('icustay_id').vent_num_imp.cumsum()
df['vent_shift'] = df.vent_num_imp.shift(-1)

df['select_peine'] = (~df['vent_shift'].isna() | (df['vent_num_imp'] == 1.0 )) & (df['vent_imp_step'] <= 18.0)
df['select_peine_step'] = df.groupby('icustay_id').select_peine.cumsum()
df['select_peine_step_rev'] = df.sort_values(['icustay_id', 'start_time'], ascending=False).groupby('icustay_id').select_peine.cumsum()

In [ ]:
df[['vent_num_imp', 'vent_num', 'select_peine', 'select_peine_step', 'select_peine_step_rev', 'start_time', 'icustay_id']]

Add the admission time for each icustay as ``start_time_icustay``

In [ ]:
df = df.merge(df.groupby('icustay_id', as_index=False)['start_time'].min(), on='icustay_id', how='left').rename(
    columns={'start_time_x': 'start_time', 'start_time_y': 'start_time_icustay'})

Calculate length of stay at the ICU: LOS ICU and length of stay in hospital: LOS Hosp

In [ ]:
# length of stay at ICU
df['los_icu'] = df.outtime - df.intime
# length of stay at hospital, LOS
df.loc[:, 'hadmittime'] = pd.to_datetime(df.hadmittime)
df.loc[:, 'hdischtime'] = pd.to_datetime(df.hdischtime)
df.loc[:, 'hadm_duration'] = df.hdischtime - df.hadmittime

In [ ]:
df.loc[:, 'mort'] = (df.mort90day == 't') | (df.hospmort == 't')

In [ ]:
df.groupby('icustay_id').first().mort.mean()

In [ ]:
(df.groupby('icustay_id').first().mort90day == 't').mean(), (df.groupby('icustay_id').first().hospmort == 't').mean()

# Analyze missing values

In [ ]:
def print_missing_pct(dataframe, variables):
    """
    Formatted print of the number of NaNs in `dataframe` for each variable in `variables`.
    """
    ljust = max(map(len, variables))
    for var in variables:
        print('{} {}'.format(var.ljust(ljust), dataframe[var].isna().mean()))

## Demographics
Analyze the missing data per patient for demographics:
* Age
* Gender
* Weight (upon admission)
* ICU readmission
* Premorbidity as calculated by exlixhauser vanWalraven score by previous hospital stay

In [ ]:
# Note that demographics do not change over time, so we can take any value. We take the first.
identifiers = ['icustay_id',]

patients = df.drop_duplicates(identifiers, keep='first')
print_missing_pct(patients, demographics)

## Vital signs
Analyze missing data across the entire dataset for vital signs. These can be composed of both lab and chart data:
* SOFA: sequential organ failure, see [code](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/severityscores/sofa.sql)
* SIRS: systemic inflammatory response syndrome (SIRS) score, see [code](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/severityscores/sirs.sql)
* GCS: glascow coma score, see [code](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/pivot/pivoted-gcs.sql)
* HR: heartrate, averaged over period
* SysBP: systolic blood pressure, averaged over period
* MeanBP: mean arterial pressure, averaged over period
* DiasBP: diastolic blood pressure, averaged over period
* Shock Index: SysBP / HeartRate, averaged over period
* RespRate: respiratory rate, averaged over period
* SpO2: blood oxygenation saturation, averaged over period
* TempC: temperature in Celsius, averaged over period

In [ ]:
print_missing_pct(df, vital_sign_vars)

## Lab measurements
All are averaged over the time span
* potassium
* sodium
* chloride
* glucose
* bun: blood urea nitrogen
* creatinine
* magnesium
* calcium
* ionizedcalcium
* carbondioxide
* bilirubin
* albumin
* hemoglobin
* wbc: white blood cell count
* platelet: platelets count
* ptt: Partial Thromboplastin Time
* pt: Prothrombin Time
* inr: International Normalized Ratio
* ph
* pao2
* paco2
* base_ excess
* bicarbonate
* lactate
* pao2fio2ratio

In [ ]:
print_missing_pct(df, lab_vars)

In [ ]:
print_missing_pct(df, lab_vars)

## Generic, non-ventilation-related treatment variables
* iv_total
* vaso_total
* urineoutput
* cum_fluid_balance: cumulated fluid balance since admission, incl. preadmission when available

These can be 'missing' due to not being applied (e.g. vasopressors)

In [ ]:
print_missing_pct(df, treatment_vars)

## Ventilation settings
* peep
* fio2
* tidal_volume
* mechvent

These can be 'missing' due to mechanical ventilation not being applied.

In [ ]:
print_missing_pct(df, vent_vars)

## Guideline vars
Some variables are in the guideline but were not in the original feature selection:
* plateau_pressure

In [ ]:
print_missing_pct(df, guideline_vars)

## Frequency analysis per variable
We estimate whether a new observation/measurement is expected in the following way:
* for every given observation, we calculate the time it took to observe.

  Specifically, we calculate the time from the previous observation OR admission.
* define a distribution of time it takes to do an observation (mean/avg?)
* estimate the expectation of a new probability based on this

In [ ]:
def tukeys_fence(series, includena=True):
    q1, q3 = series.quantile(.25), series.quantile(.75)
    iqr = q3 - q1
    lower_fence, upper_fence = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    valuefilter = (series > lower_fence) & (series < upper_fence)
    if includena:
        return valuefilter | series.isna()
    else:
        return valuefilter
    

def ceil_to_nearest_multiple(n, multiple=SAMPLE_TIME_H):
    """
    Rounds `n` up to nearest `multiple`.
    """
    return multiple * math.ceil(n / multiple)

def get_poisson_cutoff(data, quantile=.99, label='Data', sample_time=SAMPLE_TIME_H, plot=False):
    """
    Returns the value so that `quantile` % of the `data` lies below that value.
    """
    data = data - 1
    # the bins should be of integer width, because poisson is an integer distribution
    bins = np.arange(np.nanmax(data)) - 0.5
    entries, bin_edges = np.histogram(data, bins=bins, density=True)

    # calculate bin centres
    bin_middles = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    def fit_function(k, lamb):
        '''poisson function, parameter lamb is the fit parameter'''
        return poisson.pmf(k, lamb)

    # fit with curve_fit
    parameters, cov_matrix = curve_fit(fit_function, bin_middles, entries)
    cdf = poisson.cdf(np.arange(np.nanmax(data)), *parameters)
    cutoff = len(cdf[cdf < quantile])
    
    if plot:
        plt.hist(data, bins=bins, density=True, label=label)
        x_plot = np.arange(0, np.nanmax(data))
        axs = plt.plot(
            x_plot,
            fit_function(x_plot, *parameters),
            marker='o', linestyle='',
            label='poisson fit'
        )
        ax = plt.gca()
        plt.xlabel('x4 hours')
        plt.title("{}@{}: {}, max={}".format(label, quantile, cutoff*sample_time,np.nanmax(data)*sample_time))
        plt.legend()
        plt.vlines(cutoff, 0, max(entries)*1.1, colors=['black'])
        plt.show()
    
    return cutoff

def observation_durations(data, colname, groupby='icustay_id', sample_time=SAMPLE_TIME_H, scope_filter=None):
    """
    Returns the duration from admission or the previous measurements for each measurement of `colname` in `data`, for admissions identified by the `groupby` column and sample frequency of `sample_time`.
    """
    if scope_filter is None:
        scope_filter = pd.Series([True,] * data.shape[0])
    # initialize a copy of the `data` dataframe with some placeholder columns
    temp_df = pd.DataFrame.from_dict({
        'icustay_id': data.icustay_id,
        'timestamp': data.start_time,
        'timestamp_icustay': data.start_time_icustay,
        colname: data[colname],
        't_start': np.nan,
        'ages': pd.NaT,
    })
    # first set icu admission times as start time
    temp_df.loc[temp_df.timestamp == temp_df.timestamp_icustay, 't_start'] = temp_df[temp_df.timestamp == temp_df.timestamp_icustay].timestamp
    # then add timings of observations of variable in `colname`
    temp_df.loc[~temp_df[colname].isna(), 't_start'] = temp_df[~temp_df[colname].isna()].timestamp
    # now forward fill per icustay
    temp_df['t_start'] = temp_df.groupby(groupby)['t_start'].ffill()
    # and shift back by 1 time step per icustay
    temp_df['t_start'] = temp_df.groupby(groupby)['t_start'].shift()
    # and calculate the time from [previous observation|admission] until this observation
    temp_df.loc[~temp_df[colname].isna(), 'duration'] = pd.to_datetime(temp_df.timestamp) - pd.to_datetime(temp_df.t_start)
    # set time of first measurement to zero
    temp_df.loc[(temp_df['timestamp'] == temp_df['timestamp_icustay']) & ~temp_df[colname].isna()] = pd.Timedelta(0, 'hours')
    # add the length of time window to duration
    return temp_df[(~data[colname].isna() & scope_filter)]['duration'] + pd.Timedelta(sample_time, 'hours')

def observation_durations_ventstart(data, colname, groupby='icustay_id', sample_time=SAMPLE_TIME_H, scope_filter=None):
    """
    Returns the duration from admission or the previous measurements for each measurement of `colname` in `data`, for admissions identified by the `groupby` column and sample frequency of `sample_time`.
    """
    if scope_filter is None:
        scope_filter = pd.Series([True,] * data.shape[0])
    # initialize a copy of the `data` dataframe with some placeholder columns
    temp_df = pd.DataFrame.from_dict({
        'icustay_id': data.icustay_id,
        'timestamp': data.start_time,
        'timestamp_icustay': data.start_time_icustay,
        colname: data[colname],
        't_start': np.nan,
        'ages': pd.NaT,
        'mechvent_start_peine': (data.mechvent_start_peine == 1.0).fillna(False),
    })

    # first add timings of observations of variable in `colname`
    temp_df.loc[~temp_df[colname].isna(), 't_start'] = temp_df[~temp_df[colname].isna()].timestamp
    # forward fill, first pass
    temp_df.t_start.ffill()
    # then set the ventilation start as start time, if it is still missing
    temp_df.loc[temp_df.t_start.isna() & (temp_df.mechvent_start_peine), 't_start'] = temp_df[temp_df.t_start.isna() & (temp_df.mechvent_start_peine)].timestamp
    # now forward fill per icustay
    temp_df['t_start'] = temp_df.groupby(groupby)['t_start'].ffill()
    # and shift back by 1 time step per icustay
    temp_df['t_start'] = temp_df.groupby(groupby)['t_start'].shift()
    # and calculate the time from [previous observation|admission] until this observation
    temp_df.loc[~temp_df[colname].isna(), 'duration'] = pd.to_datetime(temp_df.timestamp) - pd.to_datetime(temp_df.t_start)
    # set time of first measurement to zero
    temp_df.loc[(temp_df.mechvent_start_peine) & ~temp_df[colname].isna()] = pd.Timedelta(0, 'hours')
    # add the length of time window to duration
    return temp_df[(~data[colname].isna() & scope_filter)]['duration'] + pd.Timedelta(sample_time, 'hours')

def get_cutoff_duration(data, colname, groupby='icustay_id', quantile=.99, sample_time=SAMPLE_TIME_H, plot=False, scope_filter=None):
    """
    Returns the duration so that `quantile` % of the `colname` measurements in `data` where obtained within that duration.
    """
    # obtain durations
#     durations = observation_durations(data, colname, groupby, scope_filter=scope_filter)
    durations = observation_durations_ventstart(data, colname, groupby, scope_filter=scope_filter)
    # convert to numpy array of hours
    duration_hours = (durations.dt.components.days * 24 + durations.dt.components.hours).to_numpy()
    # outlier removal using Tukey's fence
    durations = durations[tukeys_fence(durations)]
    return get_poisson_cutoff(duration_hours / sample_time, quantile, colname, plot=True) * sample_time

# Demographics
Some pre-selection demographics inspired by Table 1 in Peine et al.

In [ ]:
# unique identifier columns
identifiers = ['icustay_id', 'subject_id', 'hadm_id',]
# columns with patient demographics information
demographics = ['admission_age', 'adult_ibw', 'height', 'weight', 'icu_readmission', 'elixhauser_vanwalraven', 'hospmort', 'mort90day', 'vent_duration_h']
patients = df.drop_duplicates(identifiers + demographics, keep='first')

In [ ]:
patients.icustay_id.nunique()

In [ ]:
n_cols = len(demographics)
plt.subplots(1, len(demographics), figsize=(50,5))
for i in range(n_cols):
    plt.subplot(1, n_cols, i+1)
    if is_numeric_dtype(patients[demographics[i]]):
        sns.boxplot(data=patients, x='gender', y=demographics[i], orient='v')
    else:
        sns.countplot(data=patients, x=demographics[i], orient='v', hue='gender')

## Mechvent events
Create a single mechanical ventilation event per icustay.
Peine et. al simply sample the first recorded mechvent event.
- for each patient:
   - count each consecutive mechvent using mechvent_start_peine
   - assign this count label to each timestep at which mechvent_peine is true
   - calculate timestep before and 18 (=72/4) timesteps after onset of mechanical ventilation
   - filter by these calculated timesteps

# Imputation
Steps according to Peine et al.
* Time-windowed sample-and-hold e.g. forward-fill.

  Windows sizes were determined using a frequency analysis of the dataset, producing the frequency of production of a new datapoint (per variable)
* K-nearest neighbor imputation with imputation and SVD
* If >50% of missing data, discard (<1% of cohort)
* Test correlation between data and probability distribution of missing values for each of the 44 features.
  Here, GCS was associated with highest p-value of 0.08 -> hence distinguish missing-at-random from missing-completely-at-random and not-missed-at-random.

## Determine sample-and-hold window with frequency analysis

In [ ]:
# def ffill_window(df, var, plot, scope_filter):
#     val = get_cutoff_duration(df, var, quantile=.99, plot=plot, scope_filter=scope_filter)
#     if not plot:
#         print('.', end='') # simple measure of progress
#     return val

all_var_types = [
    vital_sign_vars,
    lab_vars,
    treatment_vars,
    vent_vars,
    guideline_vars,
]
all_vars = functools.reduce(operator.add, all_var_types)

# # NOTE: embarrasingly parallel, can be sped up with e.g. joblib.Parallel
# ffill_windows_statistical = {var: ffill_window(df, var, True, df['select_peine']) for var in all_vars}

## Clinically informed sample-and-hold windows

In [ ]:
inf = None
ffill_windows_clinical = {
    'sofa': 24 / SAMPLE_TIME_H,
    'sirs': 24 / SAMPLE_TIME_H,
    'gcs': inf,
    'heartrate': inf,
    'sysbp': inf,
    'meanbp': inf,
    'diasbp': inf,
    'shockindex': inf,
    'resprate': inf,
    'spo2': inf,
    'tempc': inf,
    'potassium': inf,
    'sodium': inf,
    'chloride': inf,
    'glucose': inf,
    'bun': inf,
    'creatinine': inf,
    'magnesium': inf,
    'calcium': inf,
    'ionizedcalcium': 8 / SAMPLE_TIME_H,
    'carbondioxide': inf,
    'bilirubin': inf,
    'albumin': inf,
    'hemoglobin': inf,
    'wbc': inf,
    'platelet': inf,
    'ptt': inf,
    'pt': inf,
    'inr': inf,
    'ph': inf,
    'pao2': inf,
    'paco2': inf,
    'base_excess': inf,
    'bicarbonate': inf,
    'lactate': inf,
    'pao2fio2ratio': inf,
    'iv_total': 8 / SAMPLE_TIME_H,
    'vaso_total': 24 / SAMPLE_TIME_H,
    'urineoutput': 8 / SAMPLE_TIME_H,
    'cum_fluid_balance': 8 / SAMPLE_TIME_H,
    'peep': 8 / SAMPLE_TIME_H,
    'fio2': 8 / SAMPLE_TIME_H,
    'tidal_volume': 8 / SAMPLE_TIME_H,
    'mechvent': 8 / SAMPLE_TIME_H,
    'plateau_pressure': 8 / SAMPLE_TIME_H
}

## Impute by sample-and-hold

In [ ]:
for var in all_vars:
    df[var + '_imp'] = df.groupby('icustay_id')[var].ffill(limit=ffill_windows_clinical[var])

imputed_vars = list(map(lambda x: x + '_imp', all_vars))

In [ ]:
max_varname_len = max(map(len, all_vars))
print("{}: {}  {}  {}".format('VARNAME'.ljust(max_varname_len), 'MISSING', 'ORIGINAL', 'IMPUTED'))
for var in all_vars:
    missing = df[var].isna().sum() / df.shape[0]
    missing_imp = df[var + '_imp'].isna().sum() / df.shape[0]
    imputed = missing - missing_imp
    print("{}: {:.4f}   {:.4f}    {:.4f}".format(
        var.ljust(max_varname_len),
        missing_imp,
        missing,
        imputed))

# Missing values selection

In [ ]:
selection = df[df['select_peine']]

max_varname_len = max(map(len, all_vars))
print("{}: {}  {}  {}".format('VARNAME'.ljust(max_varname_len), 'MISSING', 'ORIGINAL', 'IMPUTED'))
for var in all_vars:
    missing = selection[var].isna().sum() / selection.shape[0]
    missing_imp = selection[var + '_imp'].isna().sum() / selection.shape[0]
    imputed = missing - missing_imp
    print("{}: {:.4f}   {:.4f}    {:.4f}".format(
        var.ljust(max_varname_len),
        missing_imp,
        missing,
        imputed))

# Outlier removal
"Outliers were sorted out with univariate statistical approaches (Tukey's range test) and frequency analysis (90% CI)" according to Peine. Since Tukey's range test is a test for significantly different means between groups, they probably refer to Tukey's fences for outlier detection, which are defines 'non outlier' data as everything in the range ``[(first quartile - 1.5 IQR), (third quartile + 1.5 IQR)]``.


In [ ]:
boolean_numeric_vars = {'mechvent'}
tukeys_fences = []
for var in all_vars:
    if pd.api.types.is_numeric_dtype(selection[var]) and var not in boolean_numeric_vars:
        selection.loc[:, var + '_in_tukeys_fence'] = tukeys_fence(selection[var])
        tukeys_fences += [var + '_in_tukeys_fence',]
        selection.loc[~selection[var + '_in_tukeys_fence'], var] = np.NaN
        selection.loc[~selection[var + '_in_tukeys_fence'], var + '_imp'] = np.NaN

In [ ]:
# outlier removal for demographic data
ranges = {
    'weight': (25, 400),
    'admission_age': (18, 150),
    'adult_ibw': (25, 400),
}
for var in ranges:
    lower, upper = ranges[var]
    selection.loc[(selection[var] < lower) | (selection[var] > upper), var] = np.NaN

Remove patients with too many missing values

In [ ]:
to_remove = ((selection[imputed_vars + demographics].isnull().groupby(df.icustay_id).mean().mean(axis=1)) > .5)
print("Removing {} / {} = {}% patients due to >50% missing values".format(to_remove.sum(), selection.shape[0], 100*to_remove.sum()/selection.shape[0]))
selection = selection[~selection.icustay_id.isin(to_remove[to_remove].index)]
selected_patients = selection.drop_duplicates(subset='icustay_id', keep='first')

print("Selection of {} timesteps for {} patients / mech vent events".format(selection.shape[0], selected_patients.shape[0]))

In [ ]:
selection.mechvent.isna().sum() / selection.shape[0]

# Train-test-validation split

Train-test-validation split
* Train: 60%
* Test: 20%
* Validation: 20%

In [ ]:
train, test, validation = .6, .2, .2
assert (train + test + validation) == 1.0

train_split = train / (train + test)
test_split = test / (train + test)

precision = 1e8
assert (train_split + test_split) - 1.0 < precision
assert (train_split * (train + test)) - train < precision
assert (test_split * (train + test)) - test < precision

train_and_test_patients, validation_patients = train_test_split(selected_patients, test_size=0.2)
train_patients, test_patients = train_test_split(train_and_test_patients, test_size=test_split)

precision = 1.5
assert abs(train_patients.shape[0] - train * selected_patients.shape[0]) < precision
assert abs(validation_patients.shape[0] - validation * selected_patients.shape[0]) < precision
assert abs(test_patients.shape[0] - test * selected_patients.shape[0]) < precision

train_df = selection[selection.icustay_id.isin(train_patients.icustay_id)].copy()
test_df = selection[selection.icustay_id.isin(test_patients.icustay_id)].copy()
val_df = selection[selection.icustay_id.isin(validation_patients.icustay_id)].copy()

# Scale & center for imputation w/ kNN

for k-means, data has to be scaled first.
k-Means imputation

In [ ]:
# NOTE: mirror changes here in the unscaling logic in e.g. the Clustering & Analysis notebook
numeric_scaler = StandardScaler

df_stays = df.drop_duplicates(subset='icustay_id', keep='first')
scalers = {}

scaled_vars = []
for var in all_vars:
    if pd.api.types.is_numeric_dtype(selection[var]):
        ffilled_varname = var + '_imp'
        scaled_vars.append(ffilled_varname)

all_var_scaler = numeric_scaler()
scaled = all_var_scaler.fit_transform(train_df[scaled_vars])
scaled_test = all_var_scaler.transform(test_df[scaled_vars])
scaled_val = all_var_scaler.transform(val_df[scaled_vars])
scaled_varnames = [var + '_scaled' for var in all_var_scaler.get_feature_names_out()]
train_df[scaled_varnames] = scaled
test_df[scaled_varnames] = scaled_test
val_df[scaled_varnames] = scaled_val

demographics_scaled_vars = []
for var in demographics:
    if pd.api.types.is_numeric_dtype(train_df[var]):
        scaled_vars.append(var)
        demographics_scaled_vars.append(var)
        
demographics_scaler = numeric_scaler()
demographics_scaler.fit(df_stays[demographics_scaled_vars])
demo_scaled_varnames = [var + '_scaled' for var in demographics_scaler.get_feature_names_out()]
scaled = demographics_scaler.transform(train_df[demographics_scaled_vars])
scaled_test = demographics_scaler.transform(test_df[demographics_scaled_vars])
scaled_val = demographics_scaler.transform(val_df[demographics_scaled_vars])
train_df[demo_scaled_varnames] = scaled
test_df[demo_scaled_varnames] = scaled_test
val_df[demo_scaled_varnames] = scaled_val

joblib.dump(scaled_varnames, 'scalers/train/clinical_std_varnames_{}.bin'.format(seed), compress=True)
joblib.dump(all_var_scaler, "scalers/train/clinical_std_scaler_{}.bin".format(seed), compress=True)
joblib.dump(demo_scaled_varnames, 'scalers/train/demographics_std_varnames_{}.bin'.format(seed), compress=True)
joblib.dump(demographics_scaler, "scalers/train/demographics_std_scaler_{}.bin".format(seed), compress=True)

In [ ]:
# Test that scaling did not impose any NAs
assert train_df['tempc_imp'].isna().sum() == train_df['tempc_imp_scaled'].isna().sum()
assert train_df['admission_age'].isna().sum() == train_df['admission_age_scaled'].isna().sum()

assert test_df['tempc_imp'].isna().sum() == test_df['tempc_imp_scaled'].isna().sum()
assert test_df['admission_age'].isna().sum() == test_df['admission_age_scaled'].isna().sum()

In [ ]:
vars_to_impute = scaled_varnames + demo_scaled_varnames
knn_imputed_vars = list(map(lambda x: x + '_impknn', vars_to_impute))

imputer = KNNImputer() # use default of n=5, weights='uniform', metric='nan_euclidean'
imputed = imputer.fit_transform(train_df[vars_to_impute].to_numpy())
imputed_test = imputer.transform(test_df[vars_to_impute].to_numpy())
imputed_val = imputer.transform(val_df[vars_to_impute].to_numpy())

In [ ]:
train_df.loc[:, knn_imputed_vars] = imputed
val_df.loc[:,knn_imputed_vars] = imputed_val
test_df.loc[:, knn_imputed_vars] = imputed_test

In [ ]:
knn_imputed_vars

In [ ]:
# Test that scaling did not impose any NAs
assert train_df['tempc_imp'].isna().sum() > train_df['tempc_imp_scaled_impknn'].isna().sum()
assert train_df['admission_age'].isna().sum() > train_df['admission_age_scaled_impknn'].isna().sum()

assert test_df['tempc_imp'].isna().sum() > test_df['tempc_imp_scaled_impknn'].isna().sum()
assert test_df['admission_age'].isna().sum() > test_df['admission_age_scaled_impknn'].isna().sum()

assert val_df['tempc_imp'].isna().sum() > val_df['tempc_imp_scaled_impknn'].isna().sum()
assert val_df['admission_age'].isna().sum() > val_df['admission_age_scaled_impknn'].isna().sum()



In [ ]:
joblib.dump(imputer, "scalers/train/knn_imputer_{}.bin".format(seed), compress=True)

In [ ]:
def adult_ibw(row):
    if row.gender == 'M':
        return 50 + 0.91 * (row.height - 152.4)
    elif row.gender == 'F':
        return 45 + 0.91 * (row.height - 152.4)
    else:
        return np.NaN


for df_name, df in (('train', train_df), ('test', test_df), ('validation', val_df)):
    df['adult_ibw_imp_knn'] = df.apply(adult_ibw, axis=1)

In [ ]:
for df_name, df in (('train', train_df), ('test', test_df), ('validation', val_df)):
    df = df[~df['adult_ibw_imp_knn'].isna()]
    if df_name == 'train':
        df.to_csv('../mimic-data/{}_imputed_{}.csv'.format(df_name, seed))
    else:
        df.to_csv('../mimic-data/{}_ffilled_{}.csv'.format(df_name, seed))

In [ ]:
train_df_l = pd.read_csv('../mimic-data/train_imputed_{}.csv'.format(seed))
test_df_l = pd.read_csv('../mimic-data/test_ffilled_{}.csv'.format(seed))
val_df_l = pd.read_csv('../mimic-data/validation_ffilled_{}.csv'.format(seed))

train_df_l.shape[0], test_df_l.shape[0], val_df_l.shape[0]

In [ ]:
selection = pd.concat([train_df_l, test_df_l, val_df_l])
selection['los_icu'] = pd.to_timedelta(selection['los_icu'])
selection['hadm_duration'] = pd.to_timedelta(selection['hadm_duration'])
selected_patients = selection.drop_duplicates(subset='icustay_id', keep='first')

# Population demographics

In [ ]:
print("{:<30} {}".format("Included patients", selection.nunique()))
print("{:<30} {}".format("Included stays", selection.icustay_id.nunique()))
print("{:<30} {:.2f} +- ({:.2f}-{:.2f})".format(
    "Age, years median (q1-q3)",
    selected_patients.admission_age.median(),
    selected_patients.admission_age.quantile(.25),
    selected_patients.admission_age.quantile(.75)
))
print("{:<30} {:.2f} ±{:.2f}".format(
    "Adm. Weight mean ±std",
    selected_patients.weight.mean(),
    selected_patients.weight.std()
))
print("{:<30} {:.2f} ±{:.2f}".format(
    "Pred. IBW mean ±std",
    selected_patients.adult_ibw.mean(),
    selected_patients.adult_ibw.std()
))
gender_counts = selected_patients.gender.value_counts
print("{:<30} {} ({:.2f}%)".format("Female", (selected_patients.gender == 'F').sum(), 100*(selected_patients.gender == 'F').sum() / selected_patients.shape[0]))
print("{:<30} {} ({:.2f}%)".format("Male", (selected_patients.gender == 'M').sum(), 100*(selected_patients.gender == 'M').sum() / selected_patients.shape[0]))
print("{:<30} {:.2f}%".format("90-day mortality", (selected_patients.mort90day == 't').sum() / selected_patients.shape[0] / 0.01))
print("{:<30} {:.2f}%".format("hosp mortality", (selected_patients.hospmort == 't').sum() / selected_patients.shape[0] / 0.01))
los_icu_median_days = selected_patients.los_icu.astype("timedelta64[ms]").median().components.days + selected_patients.los_icu.median().components.hours / 24
los_icu_q1_days =  selected_patients.los_icu.astype("timedelta64[ms]").quantile(.25).components.days + selected_patients.los_icu.quantile(.25).components.hours / 24
los_icu_q3_days =  selected_patients.los_icu.astype("timedelta64[ms]").quantile(.75).components.days + selected_patients.los_icu.quantile(.75).components.hours / 24
print("{:<30} {:.2f} ({:.2f}-{:.2f})".format(
    "LOS ICU median (q1-q3)",
    los_icu_median_days,
    los_icu_q1_days,
    los_icu_q3_days
))
los_hosp_median_days = selected_patients.hadm_duration.astype("timedelta64[ms]").median().components.days + selected_patients.hadm_duration.astype("timedelta64[ms]").median().components.hours / 24
los_hosp_q1_days =  selected_patients.hadm_duration.quantile(.25).components.days + selected_patients.hadm_duration.astype("timedelta64[ms]").quantile(.25).components.hours / 24
los_hosp_q3_days =  selected_patients.hadm_duration.quantile(.75).components.days + selected_patients.hadm_duration.astype("timedelta64[ms]").quantile(.75).components.hours / 24
print("{:<30} {:.2f} ({:.2f}-{:.2f})".format("LOS hosp median (q1-q3)", los_hosp_median_days, los_hosp_q1_days, los_icu_q3_days))
print("{:<30} {:.1f} ±{:.2f}".format("PEEP cmH2O mean ±std", selection.peep.mean(), selection.peep.std()))
print("{:<30} {:.2f} ±{:.2f}".format("FiO2, % mean ±std", selection.fio2.mean(), selection.fio2.std()))
selection['vt_mlkg'] = selection.tidal_volume / selection.adult_ibw_imp_knn
print("{:<30} {:.2f} ±{:.2f}".format("Vt, mL/kg (PIBW) mean ±std", selection['vt_mlkg'].mean(), selection['vt_mlkg'].std()))
print("{:<30} {:.2f} ±{:.2f}".format(
    "SOFA points @ adm. mean ±std",
    selected_patients.sofa.mean(),
    selected_patients.sofa.std()
))

In [ ]:
# 35,556 decision time instances according to peine et al, Table 2a Supplementary material.
selection.shape[0], selection.shape[0] / selected_patients.shape[0]

In [ ]:
# paper reports inclusion of 35,556 decision time instances for MIMIC-III, this gives an average trajectory length of
35556 / 11443

In [ ]:
import safety

In [ ]:
selected_patients.vaso_total.describe()